In [22]:
import pandas as pd
import numpy as np
import sys
from sklearn.preprocessing import MinMaxScaler
artist_name = input()
track_name = input()

def find_song( artist_name, track_name ):
    import spotipy
    from spotipy.oauth2 import SpotifyOAuth
    import pandas as pd

    # 填入你的Spotify API金鑰
    client_id = '8ffdb014dd8c49ffa59b70a3d0d7f75a'
    client_secret = '428120b1d13c4fa18a2df4e0ba0259be'
    redirect_uri = 'http://localhost:3000'  # 這是你在應用程式設置中指定的重定向URI

    # 創建SpotifyOAuth物件進行驗證
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                                client_secret=client_secret,
                                                redirect_uri=redirect_uri
                                                ))  # scope='user-library-read''user-library-read'是API權限範圍
    from collections import defaultdict


    song_data = defaultdict()
    results = sp.search(q = 'artist_name: {} track_name: {}'.format(artist_name,
                                                       track_name), limit=1)
    if results['tracks']['items'] == []:
        return None
    
    results = results['tracks']['items'][0]

    track_id = results['id']
    audio_features = sp.audio_features(track_id)[0]
    
    song_data['artist_name'] = [artist_name]
    song_data['track_name'] = [track_name]
    song_data['popularity'] = [results['popularity']]
    song_data['year'] = [int(results['album']['release_date'].split('-')[0])]
    song_data['duration_ms'] = [results['duration_ms']]
    artist_url = results["artists"][0]["external_urls"]["spotify"]
    artist = sp.artist(artist_url)
    song_data['genre'] = [", ".join(artist["genres"])]
    
    for key, value in audio_features.items():
        song_data[key] = value

    
    return pd.DataFrame(song_data)

find_song(artist_name, track_name)

# 獲得新的歌曲資訊
song_data_df = find_song(artist_name, track_name)
song_data_df = song_data_df.rename(columns={'id': 'track_id'})

# 1.剔除instrumentalness不等於1和speechiness大於0.66的數據(待討論要寫在這裡嗎?)
if (song_data_df['instrumentalness'] != 1).any() :
    pass
else :
    print("此歌曲不符合查詢條件，請使用別首歌曲")
    sys.exit()
    
if (song_data_df['speechiness'] <= 0.66).any() :
    pass
else :
    print("此歌曲不符合查詢條件，請使用別首歌曲")
    sys.exit()

song_data_df = song_data_df.drop(['year', 'type', 'uri', 'track_href', 'analysis_url', 'time_signature', 'key', 'mode'], axis=1)
 
new_column_order = ['genre', 'artist_name', 'track_name', 'track_id', 'popularity', 'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence']

song_data_df = song_data_df[new_column_order]
song_data_df = pd.DataFrame(song_data_df)

# 2.將新資訊加入原有資料集對新資訊特徵做標準化後再將該新歌資訊提取出來
# 原資料集
df = pd.read_csv('./RawData.csv')
df = df.drop(['time_signature', 'key', 'mode'], axis=1)
df['duration_mins'] = df['duration_ms'] / 60000
song_data_df['duration_mins'] = song_data_df['duration_ms'] / 60000
df = pd.DataFrame(df)

# 3.檢查是否有重複的track_id
duplicate_ids = song_data_df[song_data_df['track_id'].isin(df['track_id'])]

if not duplicate_ids.empty:
    # 曲風特徵濃縮  
    genre_mapping = {
        'Pop': ['Pop', 'pop', 'power-pop', 'indie-pop', 'pop-film', 'k-pop', 'cantopop', 'trip-hop', 'party', 'Anime', 'anime', 'Movie', 'movie'],
        'Electronic': ['Electronic', 'electronic', 'edm', 'techno', 'trance', 'minimal-techno', 'progressive-house', 'deep-house', 'detroit-techno',
                       'house', 'club', 'breakbeat', 'hardstyle', 'drum-and-bass', 'industrial', 'chicago-house', 'hardcore', 'electro'],
        'Reggae': ['Reggae', 'reggae', 'dancehall', 'dubstep', 'dub', 'Ska', 'ska', 'Reggaeton', 'reggaeton'],
        'Rock': ['Rock', 'rock', 'alt-rock', 'punk-rock', 'psych-rock', 'hard-rock', 'rock-n-roll', 'punk', 'emo', 'goth', 'Alternative', 'alternative'],
        'Dance': ['Dance', 'dance', 'disco', 'tango', 'samba', 'salsa'],
        'Classical': ['Classical', 'classical', 'Opera', 'opera'],
        'Folk': ['Folk', 'folk', 'Country', 'country', 'forro', 'sertanejo'],
        'Blues_Jazz': ['Blues', 'blues', 'funk', 'groove', 'Jazz', 'jazz', 'Soul', 'soul', 'R&B', 'r&b'],
        'World': ['World', 'world', 'indian', 'spanish', 'swedish', 'afrobeat', 'german', 'french'],
        'Hip-Hop': ['Hip-Hop', 'hip-hop', 'Hip Hop', 'hip hop', 'Rap', 'rap'],
        'Indie': ['Indie', 'indie', 'garage', 'singer-songwriter', 'songwriter', 'show-tunes'],
        'Other': ['Comedy', 'comedy', 'sleep', 'Children’s Music', 'children’s music', "Children's Music", "children's music", 'A Capella', 'a capella', 'Soundtrack', 'soundtrack', 'metal', 'black-metal', 'death-metal', 'heavy-metal', 'metalcore', 'grindcore', 'acoustic', 'guitar', 'piano', 'romance', 'sad', 'ambient', 'chill', 'new-age', 'gospel']
    }

    matched_genres = []

    #遍歷新獲取的歌手流派
    for song_genres in df['genre']:
        matched_genre = None
        # 將多个流派拆分成列表
        song_genre_list = song_genres.split(", ")
        # 遍歷每個流派
        for song_genre in song_genre_list:
            # 遍歷 genre_mapping 字典中的每個流派
            for mapped_genre, synonyms in genre_mapping.items():
                # 檢查流派是否在當前流派的同義詞列表中
                if song_genre in synonyms:
                    matched_genre = mapped_genre
                    break  # 如果找到匹配的流派，停止循環
            if matched_genre:
                break  # 如果找到匹配的流派，停止外層循環
        matched_genres.append(matched_genre)

    df['genre'] = matched_genres

    df['loudness'] = df['loudness'] - df['loudness'].min() + 1
    df['speechiness'] = df['speechiness'] - df['speechiness'].min() + 1
    df['liveness'] = df['liveness'] - df['liveness'].min() + 1
    df['loudness'] = df['loudness'] ** 2
    df['speechiness'] = np.sqrt(df['speechiness'])
    df['liveness'] = np.sqrt(df['liveness'])
        
    genre_one_hot = pd.get_dummies(df['genre'], prefix='gn')
    df = pd.concat([df, genre_one_hot], axis=1)
#     drop原有的genre
    df = df.drop(['genre'], axis=1)
    features_to_standardize = ['loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'tempo', 'danceability', 'duration_ms', 'energy', 'valence', 'duration_mins']
    scaler = MinMaxScaler(feature_range=(-1, 1))
    df[features_to_standardize] = scaler.fit_transform(df[features_to_standardize])
    track_id_to_extract = song_data_df['track_id']
    single_song_data = df[df['track_id'] == track_id_to_extract.values[0]]
    
else:
    concat_df = pd.concat([df, song_data_df], ignore_index=True)
    
    # 曲風特徵濃縮  
    genre_mapping = {
        'Pop': ['Pop', 'pop', 'power-pop', 'indie-pop', 'pop-film', 'k-pop', 'cantopop', 'trip-hop', 'party', 'Anime', 'anime', 'Movie', 'movie'],
        'Electronic': ['Electronic', 'electronic', 'edm', 'techno', 'trance', 'minimal-techno', 'progressive-house', 'deep-house', 'detroit-techno',
                       'house', 'club', 'breakbeat', 'hardstyle', 'drum-and-bass', 'industrial', 'chicago-house', 'hardcore', 'electro'],
        'Reggae': ['Reggae', 'reggae', 'dancehall', 'dubstep', 'dub', 'Ska', 'ska', 'Reggaeton', 'reggaeton'],
        'Rock': ['Rock', 'rock', 'alt-rock', 'punk-rock', 'psych-rock', 'hard-rock', 'rock-n-roll', 'punk', 'emo', 'goth', 'Alternative', 'alternative'],
        'Dance': ['Dance', 'dance', 'disco', 'tango', 'samba', 'salsa'],
        'Classical': ['Classical', 'classical', 'Opera', 'opera'],
        'Folk': ['Folk', 'folk', 'Country', 'country', 'forro', 'sertanejo'],
        'Blues_Jazz': ['Blues', 'blues', 'funk', 'groove', 'Jazz', 'jazz', 'Soul', 'soul', 'R&B', 'r&b'],
        'World': ['World', 'world', 'indian', 'spanish', 'swedish', 'afrobeat', 'german', 'french'],
        'Hip-Hop': ['Hip-Hop', 'hip-hop', 'Hip Hop', 'hip hop', 'Rap', 'rap'],
        'Indie': ['Indie', 'indie', 'garage', 'singer-songwriter', 'songwriter', 'show-tunes'],
        'Other': ['Comedy', 'comedy', 'sleep', 'Children’s Music', 'children’s music', "Children's Music", "children's music", 'A Capella', 'a capella', 'Soundtrack', 'soundtrack', 'metal', 'black-metal', 'death-metal', 'heavy-metal', 'metalcore', 'grindcore', 'acoustic', 'guitar', 'piano', 'romance', 'sad', 'ambient', 'chill', 'new-age', 'gospel']
    }

    matched_genres = []

    #遍歷新獲取的歌手流派
    for song_genres in concat_df['genre']:
        matched_genre = None
        # 將多个流派拆分成列表
        song_genre_list = song_genres.split(", ")
        # 遍歷每個流派
        for song_genre in song_genre_list:
            # 遍歷 genre_mapping 字典中的每個流派
            for mapped_genre, synonyms in genre_mapping.items():
                # 檢查流派是否在當前流派的同義詞列表中
                if song_genre in synonyms:
                    matched_genre = mapped_genre
                    break  # 如果找到匹配的流派，停止循環
            if matched_genre:
                break  # 如果找到匹配的流派，停止外層循環
        matched_genres.append(matched_genre)

    concat_df['genre'] = matched_genres
    
    concat_df['loudness'] = concat_df['loudness'] - concat_df['loudness'].min() + 1
    concat_df['speechiness'] = concat_df['speechiness'] - concat_df['speechiness'].min() + 1
    concat_df['liveness'] = concat_df['liveness'] - concat_df['liveness'].min() + 1
    concat_df['loudness'] = concat_df['loudness'] ** 2
    concat_df['speechiness'] = np.sqrt(concat_df['speechiness'])
    concat_df['liveness'] = np.sqrt(concat_df['liveness'])
    
    genre_one_hot = pd.get_dummies(concat_df['genre'], prefix='gn')
    concat_df = pd.concat([concat_df, genre_one_hot], axis=1)
#     drop原有的genre 跟 time_signature兩個欄位
    concat_df = concat_df.drop(['genre'], axis=1)

    features_to_standardize = ['loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'tempo', 'danceability', 'duration_ms', 'energy', 'valence', 'duration_mins']
    scaler = MinMaxScaler(feature_range=(-1, 1))
    concat_df[features_to_standardize] = scaler.fit_transform(concat_df[features_to_standardize])
    track_id_to_extract = song_data_df['track_id']
    single_song_data = concat_df[concat_df['track_id'] == track_id_to_extract.values[0]]

# 4.drop掉不用的欄位
single_song_data = single_song_data.drop(['popularity', 'artist_name', 'track_name', 'duration_ms', 'track_id'], axis=1)

# 5.指定新的列順序
new_order = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_mins', 'gn_Blues_Jazz', 'gn_Classical', 'gn_Dance', 'gn_Electronic', 'gn_Folk', 'gn_Hip-Hop', 'gn_Indie', 'gn_Other', 'gn_Pop', 'gn_Reggae', 'gn_Rock', 'gn_World']

# 6.重新排列列的順序
single_song_data = single_song_data[new_order]

# 7.儲存抓取的歌的數據
single_song_data.to_csv('single_song_data.csv', index=False)

Eminem
Lose yourself
